In [22]:
import pandas as pd
from datetime import datetime


### Chargement des données

In [23]:
data = pd.read_csv('../old_data/velo.csv',sep=";")
data1=pd.read_csv("../old_data/meteo.csv",sep=",")


### Sélection des colonnes pertinentes pour l’analyse


In [24]:
colonnes_a_conserver = [
    'Numéro de boucle',
    'Jour',
    'Total',
    'Jour de la semaine',
    'Boucle de comptage',
    'Vacances'
]
df = data[colonnes_a_conserver]

df = df.rename(columns={
    'Numéro de boucle': 'code_boucle',
    'Boucle de comptage': 'nom_station',
     'Jour': 'date',
     'Jour de la semaine': 'jour_semaine',
     'Vacances': 'vacances',
     'Total': 'total',
}

)
df['date']=pd.to_datetime(df['date'])


### Création de la dimension station


In [25]:
df_station = df[['code_boucle','nom_station']]
df_station = df_station.drop_duplicates()
df_station['id_station'] = df_station.reset_index().index + 1
df_station.to_csv("../data/station.csv", index=False) 

### Création de la dimension temps 


In [26]:
df['date']=pd.to_datetime(df['date'])
df_date = df[['date','vacances']]
df_date = df_date.drop_duplicates()
df_date = df_date.reset_index(drop=True)
df_date['id_date'] = df_date.index +1

df_date['mois'] = df_date['date'].dt.month
df_date['jour'] = df_date['date'].dt.day
df_date['name_day'] = df_date['date'].dt.dayofweek + 1

df_date.to_csv("../data/temps.csv", index=False) 


###  Traitement des données météo 


In [27]:
data1=data1.rename(columns={
    'temperature_2m (°C)':'temperature',
    'precipitation (mm)':'precipitation',
    'wind_speed_10m (km/h)':'wind_speed',
    'cloud_cover (%)':'cloud_cover',
    'wind_gusts_10m (km/h)':'wind_gusts',
    'apparent_temperature (°C)':'apparent_temperature',
    'time':'date'

})

data1['date'] = pd.to_datetime(data1['date'])
data1['date'] = data1['date'].dt.date
df_meteo_jour = data1.groupby('date').max(numeric_only=True).reset_index()
df_meteo_jour['id_meteo'] = df_meteo_jour.index + 1
df_meteo_jour = df_meteo_jour[['id_meteo', 'date', 'temperature', 'precipitation', 'wind_speed', 'cloud_cover', 'wind_gusts', 'apparent_temperature']]

df_meteo_jour.to_csv("../data/meteo.csv", index=False)


###  Construction de la table de faits 
Jointure des données météo au dataset principal

In [28]:
data1['date']=pd.to_datetime(data1['date'])
df_faits_final = pd.merge(df, data1, on='date', how='left')

df_faits = df_faits_final.merge(df_station, on=['code_boucle', 'nom_station'], how='left')

df_faits['date'] = pd.to_datetime(df_faits['date'])
df_meteo_jour['date'] = pd.to_datetime(df_meteo_jour['date'])
df_date['date'] = pd.to_datetime(df_date['date'])

df_faits = df_faits.merge(df_meteo_jour[['date', 'id_meteo']], on='date', how='left')
df_faits = df_faits.merge(df_date[['date', 'id_date']], on='date', how='left')



### Sélection et nettoyage des colonnes finales


In [29]:
df_faits_clean = df_faits[['id_station', 'id_date', 'id_meteo', 'total']]

df_faits_clean = df_faits_clean.reset_index(drop=True)
df_faits_clean['id_fait'] = df_faits_clean.index + 1

df_faits_clean = df_faits_clean[['id_fait', 'id_station', 'id_date', 'id_meteo', 'total']]

df_faits_clean.to_csv("../data/fait_comptage.csv", index=False)


